Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=github%2Ftriples-posita-dyce-13%2Ftriples.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/github/triples-posita-dyce-13)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Help with custom success-based task resolution system, d12r[8,9], successes => 10, critical success and explosion on 12, triples if three matching numbers”](https://github.com/posita/dyce/discussions/13)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        # See <https://jupyterlite.readthedocs.io/en/stable/howto/configure/simple_extensions.html#avoid-the-drift-of-versions-between-the-frontend-extension-and-the-python-package>
        requirements = ["ipycanvas==0.13.2", "ipyevents==2.0.1", "ipympl==0.9.4", "ipywidgets==8.1.3", "anydyce==0.4.6"]
        try:
            import piplite ; await piplite.install(requirements, keep_going=True)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H, P
from dyce.evaluation import LimitT, PResult, explode, foreach
from numerary import RealLike
from collections import Counter
from fractions import Fraction
from typing import Iterator

def tally_successes(
    final_roll: tuple[RealLike, ...],
    outcomes_to_successes_map: dict[RealLike, int],
):
    r"""
    Returns the number of successes found in *final_roll*. This implementation does
    ***not*** require *final_roll* to be sorted. *outcomes_to_successes_map* is a
    mapping of outcomes to scalars used to compute the success tally. For example if
    *outcomes_to_successes_map* is ``{3: 1, 6: 3}``, each ``3`` in *final_roll* would
    count as one success, and each ``6`` would count as three successes.
    """
    counts = Counter(final_roll)
    exponent = any(count >= 3 for count in counts.values())
    # If we care about floating point outcomes, we would likely have to refactor this to
    # use math.isclose for determining whether our roll's outcomes appear in
    # outcomes_to_successes_map
    return (
        sum(
            counts[outcome] * scalar
            for outcome, scalar in outcomes_to_successes_map.items()
        )
        * 3**exponent
    )

def triples_homebrew(
    pool_size: int,
    pool_die: H,
    outcomes_to_successes_map: dict[RealLike, int],
    explode_limit: LimitT = Fraction(1, 100),
) -> H:
    r"""
    Expresses our homebrew mechanic and computes outcomes. *pool_size* and *pool_die*
    are used to construct a homogeneous pool for use with the mechanic.
    *outcomes_to_successes_map* and *multiple_triples* are passed to ``tally_successes``
    once a final roll is achieved. *explode_limit* is passed to our recursive function.
    """
    assert min(pool_die) >= 1  # this only works for dice with positive outcomes
    exploded_die = explode(pool_die, limit=explode_limit)
    p = pool_size @ P(exploded_die)

    def _infer_actual_roll_gen(roll: tuple[RealLike, ...]) -> Iterator[RealLike]:
        max_val = max(pool_die)
        for outcome in roll:
            if outcome > max_val:  # this outcome has exploded at some point
                total_explosions = outcome // max_val
                any_leftover_outcome = outcome % max_val
                yield from (max_val,) * total_explosions
                if any_leftover_outcome:
                    yield any_leftover_outcome
            else:
                yield outcome

    def _infer_actual_roll_and_tally(result: PResult) -> int:
        actual_roll = tuple(_infer_actual_roll_gen(result.roll))
        return tally_successes(actual_roll, outcomes_to_successes_map)

    return foreach(_infer_actual_roll_and_tally, p)

d12_ish = H(12).draw((8, 9)).lowest_terms()
d12_ish_outcomes_to_successes_map: dict[RealLike, int] = {10: 1, 11: 1, 12: 2}
print(triples_homebrew(3, d12_ish, d12_ish_outcomes_to_successes_map).format(scaled=True))

avg |    1.48
std |    2.30
var |    5.28
  0 |  34.30% |##################################################
  1 |  29.40% |##########################################
  2 |  18.48% |##########################
  3 |   9.42% |#############
  4 |   4.54% |######
  5 |   1.94% |##
  6 |   0.71% |#
  7 |   0.04% |
  9 |   0.20% |
 12 |   0.04% |
 15 |   0.06% |
 18 |   0.39% |
 21 |   0.30% |
 24 |   0.12% |
 27 |   0.04% |
 30 |   0.01% |
 33 |   0.00% |
 36 |   0.00% |
 39 |   0.00% |
 42 |   0.00% |
 45 |   0.00% |
 48 |   0.00% |
 51 |   0.00% |
 54 |   0.00% |


In [3]:
from anydyce import jupyter_visualize

pool_size = 3
h = triples_homebrew(3, d12_ish, d12_ish_outcomes_to_successes_map)
jupyter_visualize(
    (
        (f"{pool_size} @ {d12_ish}\nSuccess map: {d12_ish_outcomes_to_successes_map}\nMean: {h.mean():.2f}; Stdev: {h.stdev():.2f}", h),
    ),
    initial_burst_zero_fill_normalize=True,
)

In [4]:
# Unit tests
from dyce.evaluation import expandable
import unittest
from collections import Counter
from typing import Union

def alternate_homebrew(
    pool_size: int,
    pool_die: H,
    outcomes_to_successes_map: dict[RealLike, int],
    explode_limit: LimitT = Fraction(1, 1_000_000),
) -> H:
    r"""
    Provides a similar interface and functionality to ``triples_homebrew``. Note that
    *explode_limit* has a slightly different interpretation in this context.
    """
    p = pool_size @ P(pool_die)
    exploding_cut_short_sentinel = H({})

    def _wrapper(p: P, roll_so_far: tuple[RealLike, ...]) -> H:
        r"""
        This wrapper is here solely to allow us to pass down *roll_so_far* to
        *_explode_then_tally*.
        """

        @expandable(sentinel=exploding_cut_short_sentinel)
        def _explode_then_tally(new_result: PResult) -> Union[H, int]:
            r"""
            Our real mechanic implementation.
            """
            more_explosions = sum(
                1 for outcome in new_result.roll if outcome == max(pool_die)
            )
            updated_roll_so_far = roll_so_far + new_result.roll
            if more_explosions:
                # Update our roll so far and keep going
                res = _wrapper(more_explosions @ P(pool_die), updated_roll_so_far)
                if res != exploding_cut_short_sentinel:
                    # We got a final tally from somewhere below, so just pass that up
                    return res
            # We can be here either if we had nothing more to explode, or if we hit our
            # explosion limit, so treat the newly assembled roll as the final one, and
            # compute and return the number of successes
            return tally_successes(
                updated_roll_so_far,
                outcomes_to_successes_map,
            )

        return _explode_then_tally(p, limit=explode_limit)

    return _wrapper(p, ())

class TestTallySuccesses(unittest.TestCase):
    def test_empty_roll(self):
        assert tally_successes((), d12_ish_outcomes_to_successes_map) == 0

    def test_no_successes(self):
        assert (
            tally_successes(
                (1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7),
                d12_ish_outcomes_to_successes_map,
            )
            == 0
        )
        # Three nuthins is nuthin
        assert (
            tally_successes(
                (1, 2, 2, 3, 3, 3, 6, 6, 6, 6, 6, 6),
                d12_ish_outcomes_to_successes_map,
            )
            == 0
        )

    def test_successes(self):
        assert (
            tally_successes((1, 1, 2, 2, 3, 3, 10), d12_ish_outcomes_to_successes_map)
            == 1
        )
        assert (
            tally_successes(
                (1, 1, 2, 2, 3, 3, 10, 12), d12_ish_outcomes_to_successes_map
            )
            == 1 + 2
        )
        assert (
            tally_successes((1, 1, 2, 2, 3, 3, 11), d12_ish_outcomes_to_successes_map)
            == 1
        )
        assert (
            tally_successes(
                (1, 1, 2, 2, 3, 3, 11, 12), d12_ish_outcomes_to_successes_map
            )
            == 1 + 2
        )
        assert (
            tally_successes(
                (1, 1, 2, 2, 3, 3, 10, 11), d12_ish_outcomes_to_successes_map
            )
            == 1 + 1
        )
        assert (
            tally_successes(
                (1, 1, 2, 2, 3, 3, 10, 11, 12), d12_ish_outcomes_to_successes_map
            )
            == 1 + 1 + 2
        )
        assert (
            tally_successes((10, 10, 11, 11, 12, 12), d12_ish_outcomes_to_successes_map)
            == 1 + 1 + 1 + 1 + 2 + 2
        )
        assert (
            tally_successes(
                (2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 10),
                d12_ish_outcomes_to_successes_map,
            )
            == 3 * 1
        )

    def test_at_most_one_triple(self):
        assert (
            tally_successes((1, 1, 1, 10), d12_ish_outcomes_to_successes_map) == 3 * 1
        )
        assert (
            tally_successes((1, 1, 7, 7, 7, 11), d12_ish_outcomes_to_successes_map)
            == 3 * 1
        )
        assert tally_successes(
            (1, 1, 1, 10, 12), d12_ish_outcomes_to_successes_map
        ) == 3 * (1 + 2)

class TestMechanic(unittest.TestCase):
    def test_empty(self):
        res = triples_homebrew(0, d12_ish, d12_ish_outcomes_to_successes_map)
        assert res == H({}), f"{res!r} is not empty"

    def test_zero_success_equivalence(self):
        expected_h = 3 @ H(10).ge(8)
        expected_zero_ratio = Fraction(expected_h[0], expected_h.total)
        res_h = triples_homebrew(3, d12_ish, {10: 1, 11: 1, 12: 1})
        res_zero_ratio = Fraction(res_h[0], res_h.total)
        assert res_zero_ratio == expected_zero_ratio, f"{res_zero_ratio} is not equal to {expected_zero_ratio}"

    def test_equivalence_without_explosions(self):
        expected_h = 3 @ H(10).ge(8)
        # We're limiting explosions, but we still have tripling effects we need to claw
        # back
        res_h = triples_homebrew(3, d12_ish, {10: 1, 11: 1, 12: 1}, explode_limit=0)
        res_without_tripling_h = res_h.draw({9: res_h[9], 3: -res_h[9]})
        assert res_without_tripling_h == expected_h, f"{res_without_tripling_h} is not equal to {expected_h}"

    def test_vs_alternate(self):
        for pool_size, explode_limit in (
            (3, 1),
            (3, 2),
            (3, 3),
            (4, 1),
            (4, 2),
            (4, 3),
            (5, 1),
            (5, 2),
            (6, 1),
        ):
            expected_h = alternate_homebrew(3, d12_ish, d12_ish_outcomes_to_successes_map, explode_limit=explode_limit + 1)
            expected_equivalent_h = alternate_homebrew(3, H(10), {8: 1, 9: 1, 10: 2}, explode_limit=explode_limit + 1)
            res_h = triples_homebrew(3, d12_ish, d12_ish_outcomes_to_successes_map, explode_limit=explode_limit)
            assert expected_equivalent_h == expected_h, f"{expected_equivalent_h!r} is not equivalent to {expected_h!r}"
            assert res_h == expected_h, f"{res_h!r} is not equal to {expected_h!r}"

unittest.main(argv=[''], verbosity=2, exit=False)

test_empty (__main__.TestMechanic.test_empty) ... /tmp/ipykernel_9090/538572329.py:45: ExperimentalWarning: explode should be considered experimental and may change or disappear in future versions
  exploded_die = explode(pool_die, limit=explode_limit)
/opt/conda/lib/python3.11/site-packages/dyce/evaluation.py:934: ExperimentalWarning: expandable should be considered experimental and may change or disappear in future versions
  @expandable(sentinel=h)
/tmp/ipykernel_9090/538572329.py:64: ExperimentalWarning: foreach should be considered experimental and may change or disappear in future versions
  return foreach(_infer_actual_roll_and_tally, p)
/opt/conda/lib/python3.11/site-packages/dyce/evaluation.py:833: ExperimentalWarning: expandable should be considered experimental and may change or disappear in future versions
  return expandable(callback, sentinel=sentinel)(*args, limit=limit, **kw)
ok
test_equivalence_without_explosions (__main__.TestMechanic.test_equivalence_without_explosio